In [76]:
from torch.distributions import constraints
from pyro.infer import MCMC, NUTS, Predictive, log_likelihood
from pyro.distributions.constraints import positive_definite
import numpy as np
import pyro.distributions as dist
import pyro
import scipy as sp
import torch

ImportError: cannot import name 'log_likelihood' from 'pyro.infer' (/home/sidorenko/Projects/DASA/venv/lib/python3.8/site-packages/pyro/infer/__init__.py)

In [56]:
pyro.enable_validation(True)

In [57]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

In [69]:
N = 50
D = 3


X_train = np.random.dirichlet([3, 3, 3], (N, D))
Y_train = torch.tensor(np.argmax(np.sum(X_train, axis=1), axis=1))

X_test = np.random.dirichlet([3, 3, 3], (N, D))
Y_test = torch.tensor(np.argmax(np.sum(X_test, axis=1), axis=1))

In [59]:
Mu = pyro.param("M_mu", torch.eye(3).double())
Sigma = pyro.param("M_sigma", torch.tensor(np.tile(np.expand_dims(np.eye(3), 0), (3, 1, 1))).double())
M = dist.MultivariateNormal(Mu, Sigma)

In [60]:
M.shape(), M.batch_shape, M.event_shape

(torch.Size([3, 3]), torch.Size([3]), torch.Size([3]))

In [61]:
M_prime = M.sample()
M_prime, np.linalg.eigvals(M_prime), is_pos_def(M_prime)

(tensor([[ 1.2912, -1.3630, -1.0867],
         [-1.2597,  1.9859,  1.2798],
         [ 2.0106, -1.6547,  0.3667]], dtype=torch.float64),
 array([0.30158309+0.j        , 1.67108737+1.59716344j,
        1.67108737-1.59716344j]),
 True)

## Model

In [85]:
def model(X, Y=None):
    N, L, Dim = X.shape
    with torch.no_grad():
        M = pyro.sample("M", dist.MultivariateNormal(Mu, Sigma))
        for i in pyro.plate("sequences", N):
            x = X[i]
            if Y is None:
                y = None
            else:
                y = Y[i]
            x_i_j = torch.zeros(Dim).double()
            for j in pyro.markov(range(L)):
                x_i_j = pyro.sample(f"x_{i}_{j}", dist.Dirichlet((M @ x_i_j + x[j])))
            y_i = pyro.sample(f"y_{i}", dist.Categorical(x_i_j), obs=y)

In [71]:
kernel = NUTS(model)
mcmc = MCMC(kernel, warmup_steps=20, num_samples=30)

In [72]:
mcmc.run(X_train, Y_train)

Sample: 100%|██████████| 50/50 [1:46:47, 128.14s/it, step size=1.53e-11, acc. prob=0.228]


In [75]:
samples = mcmc.get_samples()

In [86]:
predictive = Predictive(model, samples)

In [87]:
predictive(X_test)

ValueError: The parameter concentration has invalid values
 Trace Shapes:       
  Param Sites:       
 Sample Sites:       
        M dist  3 | 3
         value  3 | 3
sequences dist    |  
         value 50 |  
    x_0_0 dist    | 3
         value    | 3
    x_0_1 dist    | 3
         value    | 3
    x_0_2 dist    | 3
         value    | 3
      y_0 dist    |  
         value    |  
    x_1_0 dist    | 3
         value    | 3
    x_1_1 dist    | 3
         value    | 3